<a href="https://colab.research.google.com/github/aadityane93/SortingLine-ml/blob/main/wavelets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tsfresh
import pywt
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score
import xgboost as xgb

In [61]:
# folder_id = '1ph5D4BDA-IysVbYA49QxOAt_jBpSa7ad' # 100hz
folder_id = '1EgezkhCQpjU8Ls9kXWPugfQ04RyV8j2m' # 400hz
# folder_id = '15mLNl3P_3Hw43q9G9KNaEwAQnvzPk5a1' # 800hz

In [62]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

folder_list = drive.ListFile({'q': "'{}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false".format(folder_id)}).GetList()


for folder in folder_list:
    folder_title = folder['title']
    print(f"Processing folder: {folder_title}")
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder['id'])}).GetList()

    for index, file in enumerate(file_list):
        file_title = file['title']
        print(f"Downloading {file_title}")
        file.GetContentFile(file_title)

Processing folder: 80_percent_train
Processing folder: 50_percent_train
Processing folder: 20_percent_train


In [63]:
# import gc
# del folder_list, file_list, folder, index, file
# gc.collect()

In [64]:
hz = 400
percent = 50

In [65]:
train_target = pd.read_csv(f'{hz}hz-train-target-{percent}percent.csv')
train_target.drop(columns=['Unnamed: 0'], inplace = True)
test_target = pd.read_csv(f'{hz}hz-test-target-{percent}percent.csv')
test_target.drop(columns=['Unnamed: 0'], inplace = True)
train_feature = pd.read_csv(f'{hz}hz-train-feature-{percent}percent.csv')
train_feature.drop(columns=['Unnamed: 0', 'Datum', 'Energie_In(Wh)','Energie_Out(Wh)', 'P_In(W)', 'P_Out(W)', 'color'], inplace = True)
test_feature = pd.read_csv(f'{hz}hz-test-feature-{percent}percent.csv')
test_feature.drop(columns=['Unnamed: 0', 'Datum', 'Energie_In(Wh)','Energie_Out(Wh)', 'P_In(W)', 'P_Out(W)', 'color' ], inplace = True)

In [66]:
train_feature

,Uhrzeit,I_In,V_In,I_Out(A),V_Out(V),Temp(°C),fileno
0,2024-09-07 16:11:11.349118:349,0.37,8.66,0.25,9.0,33.37,0
1,2024-09-07 16:11:11.352958:352,0.37,8.75,0.29,9.0,33.37,0
2,2024-09-07 16:11:11.356911:356,0.37,8.75,0.29,9.0,33.37,0
3,2024-09-07 16:11:11.357646:357,0.35,8.70,0.31,9.0,33.37,0
4,2024-09-07 16:11:11.361198:361,0.35,8.70,0.31,9.0,33.37,0
...,...,...,...,...,...,...,...
5922724,2024-09-05 14:10:04.814470:814,0.64,8.75,0.50,9.0,32.87,3499
5922725,2024-09-05 14:10:04.818397:818,0.64,8.75,0.50,9.0,32.87,3499
5922726,2024-09-05 14:10:04.819021:819,0.64,8.75,0.50,9.0,32.87,3499
5922727,2024-09-05 14:10:04.822544:822,0.56,8.70,0.54,9.0,32.87,3499


In [89]:
wavelet_type = 'db4'
decomp_level = 2

In [90]:
x_train_sorted = train_feature.sort_values(by=['fileno', 'Uhrzeit'])
x_test_sorted = test_feature.sort_values(by=['fileno', 'Uhrzeit'])

train_grouped = x_train_sorted.groupby('fileno')
test_grouped = x_test_sorted.groupby('fileno')

In [91]:
# import gc
# del x_train_sorted, x_test_sorted, train_feature, test_feature
# gc.collect()

In [92]:
def wavelet_transform(grouped):
  wavelet_df_I_In = {}
  wavelet_df_I_Out = {}
  wavelet_df_V_In = {}
  wavelet_df_V_Out = {}
  wavelet_df_Temp = {}

  wavelet_features_I_In = []
  wavelet_features_I_Out =[]
  wavelet_features_V_In =[]
  wavelet_features_V_Out =[]
  wavelet_features_Temp =[]


  for fileno, group in grouped:
      signal_I_In = group['I_In'].values
      signal_V_In = group['V_In'].values
      signal_I_Out = group['I_Out(A)'].values
      signal_V_Out = group['V_Out(V)'].values
      signal_Temp = group['Temp(°C)'].values

      # wavelet decomposition
      coeffs_I_In = pywt.wavedec(signal_I_In, wavelet_type, level=decomp_level)
      coeffs_V_In = pywt.wavedec(signal_V_In, wavelet_type, level=decomp_level)
      coeffs_I_Out = pywt.wavedec(signal_I_Out, wavelet_type, level=decomp_level)
      coeffs_V_Out = pywt.wavedec(signal_V_Out, wavelet_type, level=decomp_level)
      coeffs_Temp = pywt.wavedec(signal_Temp, wavelet_type, level=decomp_level)

  # for I_In

      # Extract features from coefficients
      features_I_In = {}
      for i, coeff in enumerate(coeffs_I_In):
          if len(coeff) == 0:
              continue
          # features: energy, entropy, variance
          features_I_In[f'I_In_wavelet_energy_level_{i}'] = np.sum(coeff**2)
          features_I_In[f'I_In_wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2 + 1e-10))
          features_I_In[f'I_In_wavelet_var_level_{i}'] = np.var(coeff)

      # Add fileno to map features back
      features_I_In['fileno'] = fileno
      wavelet_features_I_In.append(features_I_In)


  #for I_Out
      features_I_Out = {}
      for i, coeff in enumerate(coeffs_I_Out):
          if len(coeff) == 0:
              continue
          # features: energy, entropy,
          features_I_Out[f'I_Out_wavelet_energy_level_{i}'] = np.sum(coeff**2)
          features_I_Out[f'I_Out_wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2 + 1e-10))
          features_I_Out[f'I_Out_wavelet_var_level_{i}'] = np.var(coeff)

      wavelet_features_I_Out.append(features_I_Out)



  #for V_In
      features_V_In = {}
      for i, coeff in enumerate(coeffs_V_In):
          if len(coeff) == 0:
              continue
          # features: energy, entropy,
          features_V_In[f'V_In_wavelet_energy_level_{i}'] = np.sum(coeff**2)
          features_V_In[f'V_In_wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2 + 1e-10))
          features_V_In[f'V_In_wavelet_var_level_{i}'] = np.var(coeff)

      wavelet_features_V_In.append(features_V_In)


  #for V_Out
      features_V_Out = {}
      for i, coeff in enumerate(coeffs_V_Out):
          if len(coeff) == 0:
              continue
          # features: energy, entropy,
          features_V_Out[f'V_Out_wavelet_energy_level_{i}'] = np.sum(coeff**2)
          features_V_Out[f'V_Out_wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2 + 1e-10))
          features_V_Out[f'V_Out_wavelet_var_level_{i}'] = np.var(coeff)

      wavelet_features_V_Out.append(features_V_Out)

  #for Temp
      features_Temp = {}
      for i, coeff in enumerate(coeffs_Temp):
          if len(coeff) == 0:
              continue
          # features: energy, entropy,
          features_Temp[f'Temp_wavelet_energy_level_{i}'] = np.sum(coeff**2)
          features_Temp[f'Temp_wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2 + 1e-10))
          features_Temp[f'Temp_wavelet_var_level_{i}'] = np.var(coeff)

      wavelet_features_Temp.append(features_Temp)

  # Convert to DataFrame
  wavelet_df_I_In = pd.DataFrame(wavelet_features_I_In)
  wavelet_df_I_Out = pd.DataFrame(wavelet_features_I_Out)
  wavelet_df_V_In = pd.DataFrame(wavelet_features_V_In)
  wavelet_df_V_Out = pd.DataFrame(wavelet_features_V_Out)
  wavelet_df_Temp = pd.DataFrame(wavelet_features_Temp)


  wavelet_df = pd.concat([wavelet_df_I_In, wavelet_df_I_Out, wavelet_df_V_In, wavelet_df_V_Out, wavelet_df_Temp], axis=1)
  wavelet_df = wavelet_df.set_index('fileno')
  return wavelet_df


In [93]:
wavelet_df_train = wavelet_transform(train_grouped)
wavelet_df_test = wavelet_transform(test_grouped)

In [94]:
wavelet_df_train.isna().sum()

,0
I_In_wavelet_energy_level_0,0
I_In_wavelet_entropy_level_0,0
I_In_wavelet_var_level_0,0
I_In_wavelet_energy_level_1,0
I_In_wavelet_entropy_level_1,0
I_In_wavelet_var_level_1,0
I_In_wavelet_energy_level_2,0
I_In_wavelet_entropy_level_2,0
I_In_wavelet_var_level_2,0
I_Out_wavelet_energy_level_0,0


In [95]:
wavelet_df_test.isna().sum()

,0
I_In_wavelet_energy_level_0,0
I_In_wavelet_entropy_level_0,0
I_In_wavelet_var_level_0,0
I_In_wavelet_energy_level_1,0
I_In_wavelet_entropy_level_1,0
I_In_wavelet_var_level_1,0
I_In_wavelet_energy_level_2,0
I_In_wavelet_entropy_level_2,0
I_In_wavelet_var_level_2,0
I_Out_wavelet_energy_level_0,0


In [96]:
wavelet_df_test

,I_In_wavelet_energy_level_0,I_In_wavelet_entropy_level_0,I_In_wavelet_var_level_0,I_In_wavelet_energy_level_1,I_In_wavelet_entropy_level_1,I_In_wavelet_var_level_1,I_In_wavelet_energy_level_2,I_In_wavelet_entropy_level_2,I_In_wavelet_var_level_2,I_Out_wavelet_energy_level_0,...,V_Out_wavelet_var_level_2,Temp_wavelet_energy_level_0,Temp_wavelet_entropy_level_0,Temp_wavelet_var_level_0,Temp_wavelet_energy_level_1,Temp_wavelet_entropy_level_1,Temp_wavelet_var_level_1,Temp_wavelet_energy_level_2,Temp_wavelet_entropy_level_2,Temp_wavelet_var_level_2
fileno,,,,,,,,,,,,,,,,,,,,,
500,400.258762,152.006781,0.024572,1.883523,7.592274,0.002919,0.785950,4.558255,0.000612,260.542977,...,0.000110,2.946455e+06,-2.482941e+07,0.030359,14.921042,42.701092,0.023060,9.154943,33.339468,0.007129
501,403.622963,155.520469,0.023972,1.387010,6.715961,0.002122,1.124700,5.126940,0.000865,262.627593,...,0.000121,2.982253e+06,-2.513033e+07,0.034042,15.744288,44.451455,0.023981,8.588619,32.016783,0.006607
502,393.066265,138.116661,0.027988,1.362580,6.365233,0.002180,0.940918,5.095879,0.000756,256.105529,...,0.000125,2.856588e+06,-2.407361e+07,0.029951,13.949955,38.470191,0.022318,7.493789,27.975942,0.006024
503,371.553896,132.996956,0.027576,1.776517,7.296069,0.003001,0.894870,4.987599,0.000760,238.415018,...,0.000120,2.708526e+06,-2.282859e+07,0.025705,10.170776,29.557984,0.017178,7.397632,25.816739,0.006285
504,363.919622,122.982806,0.029683,1.887232,6.642973,0.003289,0.790241,4.384953,0.000694,231.617316,...,0.000108,2.618165e+06,-2.206356e+07,0.030049,13.021583,37.308515,0.022696,8.158601,29.274729,0.007163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,344.016415,297.325059,0.012300,1.684758,8.758630,0.001912,0.727334,4.729560,0.000414,363.699672,...,0.000131,5.316708e+06,-4.628363e+07,0.032069,18.326441,54.849974,0.020791,11.369293,43.837249,0.006475
3996,342.920936,292.559850,0.012362,1.880276,9.255499,0.002168,0.960701,5.577812,0.000556,353.565775,...,0.000137,5.231536e+06,-4.554150e+07,0.029649,17.086914,52.068872,0.019698,10.965304,42.042888,0.006346
3997,325.133996,277.065855,0.013157,1.624316,8.010316,0.001963,0.708039,4.572233,0.000430,338.913825,...,0.000129,4.996050e+06,-4.349743e+07,0.030535,15.629082,46.630058,0.018791,9.821019,37.370718,0.005963


In [97]:
train_target

,color
0,0
1,0
2,0
3,0
4,0
...,...
1995,3
1996,3
1997,3
1998,3


In [98]:
model2 = RandomForestClassifier()
model2.fit(wavelet_df_train, train_target)
y_pred = model2.predict(wavelet_df_test)
print(classification_report(test_target, y_pred))
accura = sklearn.metrics.accuracy_score(test_target, y_pred)
print(f"Accuracy: {accura:.4f}")
precision_score_ = sklearn.metrics.precision_score(test_target, y_pred, average='weighted')
print(f"Precision: {precision_score_:.4f}")
cm = confusion_matrix(test_target, y_pred)
print("Confusion Matrix:")
print(cm)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


              precision    recall  f1-score   support

           0       0.99      0.97      0.98       500
           1       0.72      0.92      0.81       500
           2       0.91      0.91      0.91       500
           3       0.97      0.72      0.83       500

    accuracy                           0.88      2000
   macro avg       0.90      0.88      0.88      2000
weighted avg       0.90      0.88      0.88      2000

Accuracy: 0.8815
Precision: 0.8995
Confusion Matrix:
[[484   1  15   0]
 [  0 461  29  10]
 [  3  41 456   0]
 [  0 138   0 362]]


In [99]:
final_model = xgb.XGBClassifier(
    # objective="multi:softmax",
    # random_state=42,
    alpha=10,
    reg_lambda=1,
    # max_depth=10,
    min_child_weight=10,
    # subsample=0.8,
    colsample_bytree=0.8
)

final_model.fit(wavelet_df_train, train_target)

y_pred_final = final_model.predict(wavelet_df_test)

final_accuracy = accuracy_score(test_target, y_pred_final)
final_precision_ = precision_score(test_target, y_pred_final, average='weighted')
final_cm = confusion_matrix(test_target, y_pred_final)
print(classification_report(test_target, y_pred_final))
print(f"Accuracy: {final_accuracy:.4f}")
print(f"Precision: {final_precision_:.4f}")
print("Confusion Matrix:")
print(final_cm)

              precision    recall  f1-score   support

           0       1.00      0.97      0.98       500
           1       0.73      0.88      0.80       500
           2       0.88      0.96      0.92       500
           3       0.97      0.70      0.82       500

    accuracy                           0.88      2000
   macro avg       0.89      0.88      0.88      2000
weighted avg       0.89      0.88      0.88      2000

Accuracy: 0.8775
Precision: 0.8926
Confusion Matrix:
[[483   0  17   0]
 [  0 441  48  11]
 [  2  19 479   0]
 [  0 146   2 352]]
